In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays 
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   

include("Params_PLD.jl")
include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
include("Estimation_PLD_Fast.jl")
include("Models_PLD.jl")
include("Evaluation_PLD.jl")
include("Implement_All_Methods_PLD.jl")
include("Figures_PLD.jl")

hist_profit_distribution (generic function with 1 method)

In [2]:
data_dir = "/Users/zhangxun/Codes/Data/Product_Line_Design/Model_Mis_Wang_Qi_Shen_N=3_N_x=8_N_u=10_N_nonzero=20_dr=2.0_seed=2_Same_Util_Para/"
Params = load(string(data_dir, "Params.jld2"));

In [3]:
# Params = get_Wang_Qi_Shen_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
norm_bounds = Params["norm_bounds"]
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_para_Input = Params["coef_this"] # coefficient for data generation
# coef_Wang_Qi_Shen = Params["coef_Wang_Qi_Shen"] # coefficient for Wang, Qi, Shen data generation

(alp0_lb = 1.0, alp0_ub = 2.0, alp_lb = -1.0, alp_ub = 0.0, beta_lb = -2.0, beta_ub = 2.0, A_lb = -2.0, A_ub = 2.0, r0_lb = 0.0, r0_ub = 1.0, r_lb = 0.0, r_ub = 0.1)

In [4]:
function output_results(S_train,lambda,data_dir,instances,fig_display)
    Input_Data = load(string(data_dir, "Input_Data_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_True_All = load(string(data_dir, "RST_True_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_ETO_All = load(string(data_dir, "RST_ETO_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_RO_All = load(string(data_dir, "RST_RO_S=$(S_train)_lambda=$(lambda).jld2"));

    gamma_list = sort([parse(Float64, split(k, "=")[end]) for k in keys(RST_RO_All["ins=1"])])
    gamma_list = gamma_list
    # println("Gamma list: ", gamma_list)

    obj_True, obj_ETO, obj_RO = obtain_obj(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),obj True:",round.(mean(obj_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj ETO:",round.(mean(obj_ETO),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj RO:",round.(mean(obj_RO,dims=1),digits=4))
    println()
    profit_True, profit_ETO, profit_RO = obtain_profits(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),profit True:",round.(mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit ETO/True:",round.(mean(profit_ETO)/mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/True:",round.(mean(profit_RO,dims=1)./mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/ETO:",round.(mean(profit_RO,dims=1)./mean(profit_ETO),digits=4))
    # fig_name = string(data_dir, "RPLD_vs_ETOPLD_S=$(S_train)_lambda=$lambda.pdf")
    # include_std = false
    # line_plot_RPLD_vs_ETOPLD(profit_ETO,profit_RO,gamma_list,include_std,fig_name,fig_display)
    return obj_True, obj_ETO, obj_RO, profit_True, profit_ETO, profit_RO
end

output_results (generic function with 1 method)

In [5]:
function plot_box(S_train,lambda,data_dir,chosen_indices,is_display)
    Input_Data = load(string(data_dir, "Input_Data_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_True_All = load(string(data_dir, "RST_True_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_ETO_All = load(string(data_dir, "RST_ETO_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_RO_All = load(string(data_dir, "RST_RO_S=$(S_train)_lambda=$(lambda).jld2"));

    gamma_list = sort([parse(Float64, split(k, "=")[end]) for k in keys(RST_RO_All["ins=1"])])
    gamma_list = gamma_list
    # println("Gamma list: ", gamma_list)

    obj_True, obj_ETO, obj_RO = obtain_obj(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),obj True:",round.(mean(obj_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj ETO:",round.(mean(obj_ETO),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj RO:",round.(mean(obj_RO,dims=1),digits=4))
    println()
    profit_True, profit_ETO, profit_RO = obtain_profits(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),profit True:",round.(mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit ETO/True:",round.(mean(profit_ETO)/mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/True:",round.(mean(profit_RO,dims=1)./mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/ETO:",round.(mean(profit_RO,dims=1)./mean(profit_ETO),digits=4))

    Profit_ETO_All_Ins = profit_ETO./mean(profit_ETO)
    Profit_RO_All_Ins = Dict();
    for g_index in 1:length(gamma_list)
        gamma=gamma_list[g_index]
        Profit_RO_All_Ins["gamma=$(gamma)"] = profit_RO[:,g_index]./mean(profit_ETO)
    end
    
    gamma_chosen = gamma_list[chosen_indices]
    data = [Profit_ETO_All_Ins, [Profit_RO_All_Ins["gamma=$(gamma)"] for gamma in gamma_chosen]...]
    labels = ["ETO"; ["RO($gamma)" for gamma in gamma_chosen]]
    fig_name = string(data_dir, "Boxplot_RPLD_vs_ETOPLD_S=$(S_train)_lambda=$(lambda)_Normalize.pdf")
    boxplot_RPLD_vs_ETOPLD(data,labels,fig_name,is_display)

end

plot_box (generic function with 1 method)

### Lambda = 0.001

In [6]:
# S_train_list = Params["S_train_all"] # training data size
# is_ridge = Params["is_ridge"] # whether to use ridge regression

In [7]:
# S_train = 500
# lambda = 0.001
# obj_True, obj_ETO, obj_RO, profit_True, profit_ETO, profit_RO = output_results(S_train,lambda,data_dir,instances,false);

### Model Misspecification

In [8]:
data_dir = "/Users/zhangxun/Codes/Data/Product_Line_Design/Model_Mis_Wang_Qi_Shen_N=3_N_x=8_N_u=10_N_nonzero=20_dr=2.0_seed=2_Same_Util_Para/"
Params = load(string(data_dir, "Params.jld2"));

In [9]:
# Params = get_Wang_Qi_Shen_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
norm_bounds = Params["norm_bounds"]
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_para_Input = Params["coef_this"] # coefficient for data generation
# coef_Wang_Qi_Shen = Params["coef_Wang_Qi_Shen"] # coefficient for Wang, Qi, Shen data generation

(alp0_lb = 1.0, alp0_ub = 2.0, alp_lb = -1.0, alp_ub = 0.0, beta_lb = -2.0, beta_ub = 2.0, A_lb = -2.0, A_ub = 2.0, r0_lb = 0.0, r0_ub = 1.0, r_lb = 0.0, r_ub = 0.1)

In [10]:
function output_results_Mis(num_c,S_train,lambda,data_dir,instances,fig_display)
    Input_Data = load(string(data_dir, "Input_Data_S=$(S_train)_lambda=$(lambda)_num_c=$(num_c).jld2"));
    RST_True_All = load(string(data_dir, "RST_True_S=$(S_train)_lambda=$(lambda)_num_c=$(num_c).jld2"));
    RST_ETO_All = load(string(data_dir, "RST_ETO_S=$(S_train)_lambda=$(lambda)_num_c=$(num_c).jld2"));
    RST_RO_All = load(string(data_dir, "RST_RO_S=$(S_train)_lambda=$(lambda)_num_c=$(num_c).jld2"));

    gamma_list = sort([parse(Float64, split(k, "=")[end]) for k in keys(RST_RO_All["ins=1"])])
    gamma_list = gamma_list
    # println("Gamma list: ", gamma_list)

    obj_True, obj_ETO, obj_RO = obtain_obj(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    # println("S=$(S_train),lambda=$(lambda),num_c = $(num_c),obj True:",round.(mean(obj_True),digits=4))
    # println("S=$(S_train),lambda=$(lambda),num_c = $(num_c),obj ETO:",round.(mean(obj_ETO),digits=4))
    # println("S=$(S_train),lambda=$(lambda),num_c = $(num_c),obj RO:",round.(mean(obj_RO,dims=1),digits=4))
    println()
    profit_True, profit_ETO, profit_RO = obtain_profits(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    # println("S=$(S_train),lambda=$(lambda),num_c = $(num_c),profit True:",round.(mean(profit_True),digits=4))
    # println("S=$(S_train),lambda=$(lambda),num_c = $(num_c),profit ETO/True:",round.(mean(profit_ETO)/mean(profit_True),digits=4))
    # println("S=$(S_train),lambda=$(lambda),num_c = $(num_c),profit RO/True:",round.(mean(profit_RO,dims=1)./mean(profit_True),digits=4))
    
    println("S=$(S_train),lambda=$(lambda),num_c = $(num_c),profit RO/ETO:",round.(mean(profit_RO,dims=1)./mean(profit_ETO),digits=4))
    # fig_name = string(data_dir, "RPLD_vs_ETOPLD_S=$(S_train)_lambda=$lambda.pdf")
    # include_std = false
    # line_plot_RPLD_vs_ETOPLD(profit_ETO,profit_RO,gamma_list,include_std,fig_name,fig_display)
    return obj_True, obj_ETO, obj_RO, profit_True, profit_ETO, profit_RO
end

output_results_Mis (generic function with 1 method)

In [11]:
S_train = 500
lambda = 0.001
for num_c in [1,2,4]
    output_results_Mis(num_c,S_train,lambda,data_dir,instances,false);
end


S=500,lambda=0.001,num_c = 1,profit RO/ETO:[1.0 1.0033 1.0045 0.9988 0.9937 0.9907 0.9886]

S=500,lambda=0.001,num_c = 2,profit RO/ETO:[1.0 1.0014 0.9913 0.9591 0.9158 0.8847 0.8501]

S=500,lambda=0.001,num_c = 4,profit RO/ETO:[1.0 1.0063 1.0103 1.0087 1.0083 1.0051 1.0023]


## lambda = 0.01

In [12]:
data_dir = "/Users/zhangxun/Codes/Data/Product_Line_Design/0108_Model_Mis_Wang_Qi_Shen_N=3_N_x=8_N_u=10_N_nonzero=20_dr=2.0_seed=2_Same_Util_Para/"
Params = load(string(data_dir, "Params.jld2"));

In [13]:
# Params = get_Wang_Qi_Shen_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
norm_bounds = Params["norm_bounds"]
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_para_Input = Params["coef_this"] # coefficient for data generation
# coef_Wang_Qi_Shen = Params["coef_Wang_Qi_Shen"] # coefficient for Wang, Qi, Shen data generation

(alp0_lb = 1.0, alp0_ub = 2.0, alp_lb = -1.0, alp_ub = 0.0, beta_lb = -2.0, beta_ub = 2.0, A_lb = -2.0, A_ub = 2.0, r0_lb = 0.0, r0_ub = 1.0, r_lb = 0.0, r_ub = 0.1)

In [14]:
S_train = 500
lambda = 0.01
for num_c in [1,2,4]
    output_results_Mis(num_c,S_train,lambda,data_dir,instances,false);
end


S=500,lambda=0.01,num_c = 1,profit RO/ETO:[1.0 1.0032 1.0042 0.9968 0.9921 0.9878 0.984]

S=500,lambda=0.01,num_c = 2,profit RO/ETO:[1.0 1.005 1.015 0.9844 0.9269 0.885 0.8542]

S=500,lambda=0.01,num_c = 4,profit RO/ETO:[1.0002 1.0047 1.0037 1.0023 0.9981 0.9921 0.9887]


## Backup

In [15]:
data_dir = "/Users/zhangxun/Codes/Data/Product_Line_Design/Backup_Model_Mis_Wang_Qi_Shen_N=3_N_x=8_N_u=10_N_nonzero=20_dr=2.0_seed=2_Same_Util_Para/"
Params = load(string(data_dir, "Params.jld2"));

In [16]:
S_train = 500
lambda = 0.001
for num_c in [2,4,8]
    output_results_Mis(num_c,S_train,lambda,data_dir,instances,false);
end


S=500,lambda=0.001,num_c = 2,profit RO/ETO:[0.9997 1.0262 1.0307 1.0404 1.0345 1.0258 1.0158]

S=500,lambda=0.001,num_c = 4,profit RO/ETO:[1.0 1.0163 1.0139 1.0041 0.9936 0.9718 0.9499]

S=500,lambda=0.001,num_c = 8,profit RO/ETO:[1.0 1.0137 1.0248 1.0268 1.0148 1.0063 1.0012]
